In [ ]:
# Workaround for training output not visible in JupyterNotebook https://github.com/microsoft/vscode-jupyter/issues/13163
from IPython.display import clear_output, DisplayHandle
def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
! pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz


In [ ]:
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

## Tabular Modeling

Take data in the form of a table, like CSV/spreadsheet. Predict the value in one column based on values in other columns. 
Some data are numerical like age, kilometers. Some data are strings like name, city. The numerical data can be fed directly to models, with optional preprocessing. The other columns need converting into numbers. These kinds of values are called categorical variables. Age, distance, are continuous variables. Categorical varibles can be numerical, or stored as numbers, but don't necessarily have meaning when multiplied. 

Modern machine learning has two key techniques: ensembles of decision trees (eg random forests, gradient boosting) for strucutred data. And multilayered neural networks (shallow/deep learning) for unstructured data. Decision tree ensembles are easy and fast to train and interpret, less tuning required, and are good at tabular data. The exceptions are high cardinality categorical variables (eg name, with lots of discrete values) and data like plain text... where deep learning may be preferred. The preferred libraries for decision trees are scikit and pandas. 

Blue Book for bulldozers dataset. The goal is to predict the sale price of a particular piece of heavy equipment at an auction based on usage, equipment type, configuration. 


In [ ]:
!pip install kaggle

In [ ]:
creds = ''
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [ ]:
comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)
path

In [ ]:
Path.BASE_PATH = path

In [ ]:
from kaggle import api

if not path.exists():
    path.mkdir(parents=true)

api.competition_download_cli(comp, path=path)
shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

path.ls(file_type='text')